In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from typing import List

MYDIR = "./../../ResearchProposal/bea_gov/personal_income/"

print(os.path)
myFiles = os.listdir(MYDIR)
print(myFiles)

print("Done")

myFile = MYDIR + "personal_income_by_county_2015_2018.xlsx"

<module 'ntpath' from 'C:\\Users\\alexg\\Anaconda3\\lib\\ntpath.py'>
['personal_income_by_county_2015_2018.pdf', 'personal_income_by_county_2015_2018.xlsx', '~$personal_income_by_county_2015_2018.xlsx']
Done


In [2]:
"""
Create column for states.
The way to identify state is: it is the first row after NaN in "aggregation_level"
It is followed by all its counties.
Between the last county of the state and the next state, there is another NaN
"""
def setState(myDF: pd.DataFrame, verbose: bool = False) -> pd.DataFrame:
    """
    get state into a separate column
    """
    myDF["state"] = ""

    states = myDF[myDF.aggregation_level == "nan"]
    if verbose:
        print(states.index)
    for rr in list(states.index): #range((len(myDF)-1)):
        if verbose:
            print(f"""{rr}: {myDF.loc[rr + 1]["aggregation_level"]}""")
            
        myDF.at[rr + 1, "state"] = myDF.loc[rr + 1]["aggregation_level"]

    for rr in range(1, (len(myDF)-1)):
        if rr - 1 in list(states.index):
            continue
        else:
            myDF.at[rr, "state"] = myDF.loc[rr-1, "state"]
    
    myDF.drop(states.index, inplace=True)
    
    myDF.at[0, "state"] = myDF.loc[0, "aggregation_level"]
    
    return myDF

"""
Split the GDP data from the gdpDF
"""
def splitOutIncomedata(incomeDF: pd.DataFrame,
                    verbose: bool = False) -> pd.DataFrame:

    
    cols1 = list(incomeDF.loc[0])
    cols2 = list(incomeDF.loc[1])
    cols3 = [str(cc).replace(".0", "") for cc in list(incomeDF.loc[2])]

    """
    We are going to handle this as two dataframes: gdp and percent change
    We are not interested in percent change; only get the gdp columns
    """
    cols = list(incomeDF.columns)
    rank_cols = [cc for cc in range(len(cols2)) if "Rank" in str(cols2[cc]) ]
    rank_cols

    income_cols = cols[:rank_cols[0]]

    if verbose:
        print(cols1)
        print(cols2)
        print(cols3)
        print(income_cols)

    """
    Separate gdp from percent change
    """
    income = incomeDF[income_cols].copy()
    
    """
    Handle column names
    """
    income_col_names = [str(cols1[cc]) + "_" + str(cols2[cc]) + "_" + str(cols3[cc]) for cc in range(len(income_cols))]
    income.columns = income_col_names
    income.drop(range(3), inplace=True)
    income.rename(columns={"nan_nan_nan": "aggregation_level", 
                        "Per capita personal income1_Dollars_2016": "nan_nan_2016"},
               inplace=True)
    income.columns = [cc.replace("nan_nan_", "income_usd_") for cc in income.columns]
    
    if verbose:
        print(income.columns)
    
    """
    Set type for "aggregation_level" to "str"
    """
    income["aggregation_level"] = income["aggregation_level"].astype("str")
    income.reset_index(drop=True, inplace=True)
        
    """
    get state into a separate column
    """
    incomefinal = setState(income.copy(), verbose=verbose)
    incomefinal.dropna(subset=["income_usd_2016"], inplace=True)

    """
    get log of the gdp numbers
    """
    for cc in incomefinal.columns:
        if "income_usd_" in cc:
            print(cc)
            incomefinal[cc] = incomefinal[cc].astype("float64")
            incomefinal[cc + "_log"] = np.log10(incomefinal[cc])


    return incomefinal

In [3]:
incomeDF = pd.read_excel(myFile, sheet_name="Table", na_values="", keep_default_na=False)

num_rows = len(incomeDF)
print(num_rows)
incomeDF.head()

cols = list(incomeDF.columns)
print(cols[0])
incomeDF.rename(columns={cols[0]: "aggregation_level"}, inplace=True)
incomeDF.head()

incomeDF.tail()

3223
Table 1. Per Capita Personal Income by County, 2016 - 2018


,aggregation_level,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
3218,Washakie,43737,44930.0,48184.0,15,2.7,7.2,8
3219,Weston,42585,42816.0,44737.0,17,0.5,4.5,19
3220,1. Per capita personal income was computed usi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3221,2. Virginia combination areas consist of one o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3222,Source: U.S. Bureau of Economic Analysis,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
incomeDF.head()

,aggregation_level,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,Per capita personal income1,NaN,NaN,NaN,Percent change from preceding period,NaN,NaN
1,NaN,Dollars,NaN,NaN,Rank in State,Percent change,NaN,Rank in State
2,NaN,2016,2017.0,2018.0,2018,2017,2018.0,2018
3,United States,49870,51885.0,54446.0,--,4,4.9,--
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
incomeFinal = splitOutIncomedata(incomeDF, verbose=True)

[nan, 'Per capita personal income1', nan, nan, nan, 'Percent change from preceding period', nan, nan]
[nan, 'Dollars', nan, nan, 'Rank in State', 'Percent change', nan, 'Rank in State']
['nan', '2016', '2017', '2018', '2018', '2017', '2018', '2018']
['aggregation_level', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']
Index(['aggregation_level', 'income_usd_2016', 'income_usd_2017',
       'income_usd_2018'],
      dtype='object')
Int64Index([   1,   70,  101,  118,  195,  255,  321,  331,  336,  338,  407,
             568,  574,  620,  724,  818,  919, 1026, 1148, 1214, 1232, 1258,
            1274, 1359, 1448, 1532, 1649, 1707, 1802, 1821, 1833, 1856, 1891,
            1955, 2057, 2112, 2202, 2281, 2319, 2388, 2395, 2443, 2511, 2608,
            2864, 2895, 2911, 3020, 3061, 3118, 3192],
           dtype='int64')
1: Alabama
70: Alaska
101: Arizona
118: Arkansas
195: California
255: Colorado
321: Connecticut
331: Delaware
336: District of Columbia
338: Florida
407: Georgia
568: Hawaii
574:

In [7]:
incomeFinal.tail()

,aggregation_level,income_usd_2016,income_usd_2017,income_usd_2018,state,income_usd_2016_log,income_usd_2017_log,income_usd_2018_log
3212,Sweetwater,47291.0,50354.0,53145.0,Wyoming,4.674778,4.702034,4.725462
3213,Teton,214020.0,227753.0,251728.0,Wyoming,5.330454,5.357464,5.400932
3214,Uinta,37634.0,38154.0,40280.0,Wyoming,4.575580,4.581540,4.605089
3215,Washakie,43737.0,44930.0,48184.0,Wyoming,4.640849,4.652536,4.682903
3216,Weston,42585.0,42816.0,44737.0,Wyoming,4.629257,4.631606,4.650667


In [8]:
incomeFinal.head()

,aggregation_level,income_usd_2016,income_usd_2017,income_usd_2018,state,income_usd_2016_log,income_usd_2017_log,income_usd_2018_log
0,United States,49870.0,51885.0,54446.0,United States,4.697839,4.715042,4.735966
2,Alabama,39224.0,40467.0,42238.0,Alabama,4.593552,4.607101,4.625703
3,Autauga,39561.0,40450.0,41618.0,Alabama,4.597267,4.606919,4.619281
4,Baldwin,42907.0,43989.0,45596.0,Alabama,4.632528,4.643344,4.658927
5,Barbour,31595.0,33048.0,35199.0,Alabama,4.499618,4.519145,4.546530


In [10]:
incomeCSV = MYDIR + "personal_income_ready_to_analyze.csv"
incomeFinal.to_csv(incomeCSV, index=False)
print(f"""Saved incomeFinal ({len(incomeFinal)} rows) to {incomeCSV}""")

Saved incomeFinal (3164 rows) to ./../../ResearchProposal/bea_gov/personal_income/personal_income_ready_to_analyze.csv
